# Base de Dados

Para a realização deste trabalho utilizamos dados disponibilizados publicamente pela Organização Mundial da Saúde (OMS). Nessa primeira seção iremos prover uma descrição básica dos dados e realizar uma limpeza dos mesmos, gerando um conjunto de dados apropriado para exploração nas demais seções.

In [2]:
import pandas as pd
import re

## WHO TB burden estimates

**Descrição:** Base de dados completa gerada pela OMS para o *Global Tuberculosis Report* (Relatório Global sobre a Tuberculose). Inclui estimativas de mortalidade, incidência de casos, taxa de mortalidade, cobertura de tratamento, status de HIV e diversas outras informações clínicas, para todos os países incluídos no relatório, a partir do ano 2000.

Inclui-se um dicionário de dados contendo os nomes das variáveis do conjunto de dados e suas definições.

**Fonte:** [WHO TB burden estimates](https://www.who.int/tb/country/data/download/en/)

**Arquivos:** A base de dados disponibilizada encontra-se em `/data/raw/TB_burden_countries_2020-11-01.csv`, e o dicionário de dados em `/data/raw/TB_data_dictionary_2020-11-01`.

In [4]:
WHO_TB = pd.read_csv('data/raw/TB_burden_countries_2020-11-01.csv')
WHO_TB.head()

,country,iso2,iso3,iso_numeric,g_whoregion,year,e_pop_num,e_inc_100k,e_inc_100k_lo,e_inc_100k_hi,e_inc_num,e_inc_num_lo,e_inc_num_hi,e_tbhiv_prct,e_tbhiv_prct_lo,e_tbhiv_prct_hi,e_inc_tbhiv_100k,e_inc_tbhiv_100k_lo,e_inc_tbhiv_100k_hi,e_inc_tbhiv_num,e_inc_tbhiv_num_lo,e_inc_tbhiv_num_hi,e_mort_exc_tbhiv_100k,e_mort_exc_tbhiv_100k_lo,e_mort_exc_tbhiv_100k_hi,e_mort_exc_tbhiv_num,e_mort_exc_tbhiv_num_lo,e_mort_exc_tbhiv_num_hi,e_mort_tbhiv_100k,e_mort_tbhiv_100k_lo,e_mort_tbhiv_100k_hi,e_mort_tbhiv_num,e_mort_tbhiv_num_lo,e_mort_tbhiv_num_hi,e_mort_100k,e_mort_100k_lo,e_mort_100k_hi,e_mort_num,e_mort_num_lo,e_mort_num_hi,cfr,cfr_lo,cfr_hi,cfr_pct,cfr_pct_lo,cfr_pct_hi,c_newinc_100k,c_cdr,c_cdr_lo,c_cdr_hi
0,Afghanistan,AF,AFG,4,EMR,2000,20779957,190.0,123.0,271.0,39000,26000,56000,0.28,0.08,0.62,0.54,0.12,1.3,110.0,25.0,260.0,68.0,39.0,104.0,14000.0,8100.0,22000.0,0.17,0.00,0.71,35.0,0.0,150.0,68.0,39.0,104.0,14000.0,8100.0,22000.0,NaN,NaN,NaN,NaN,NaN,NaN,34.0,18.0,13.0,28.0
1,Afghanistan,AF,AFG,4,EMR,2001,21606992,189.0,123.0,271.0,41000,26000,58000,0.26,0.07,0.59,0.50,0.11,1.2,110.0,23.0,250.0,63.0,36.0,96.0,14000.0,7800.0,21000.0,0.30,0.06,0.72,65.0,14.0,160.0,63.0,36.0,96.0,14000.0,7900.0,21000.0,NaN,NaN,NaN,NaN,NaN,NaN,47.0,25.0,17.0,38.0
2,Afghanistan,AF,AFG,4,EMR,2002,22600774,189.0,122.0,270.0,43000,28000,61000,0.26,0.06,0.59,0.49,0.10,1.2,110.0,23.0,270.0,57.0,33.0,87.0,13000.0,7500.0,20000.0,0.27,0.06,0.66,62.0,13.0,150.0,57.0,33.0,87.0,13000.0,7500.0,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,61.0,32.0,23.0,50.0
3,Afghanistan,AF,AFG,4,EMR,2003,23680871,189.0,122.0,270.0,45000,29000,64000,0.27,0.07,0.62,0.51,0.10,1.2,120.0,25.0,300.0,58.0,34.0,88.0,14000.0,7900.0,21000.0,0.29,0.06,0.71,69.0,14.0,170.0,58.0,34.0,89.0,14000.0,8000.0,21000.0,NaN,NaN,NaN,NaN,NaN,NaN,58.0,31.0,22.0,48.0
4,Afghanistan,AF,AFG,4,EMR,2004,24726689,189.0,122.0,270.0,47000,30000,67000,0.30,0.07,0.69,0.56,0.11,1.4,140.0,28.0,340.0,51.0,30.0,78.0,13000.0,7400.0,19000.0,0.29,0.06,0.70,71.0,14.0,170.0,52.0,30.0,78.0,13000.0,7500.0,19000.0,NaN,NaN,NaN,NaN,NaN,NaN,74.0,39.0,28.0,61.0


## Limpeza de Dados
A base da OMS é extensa, seu dicionário de dados listando mais de 500 variáveis, muitas desnecessárias para a execução deste estudo. Além disso, possuem dados de todos os países, o que excede o escopo proposto. Aqui realizaremos um aparo da base de dados, facilitando a visualização e uso dos mesmos e restringindo os dados aos países da América do Sul.

In [28]:
# filtering data
## not including overseas territories such as falkland islands or french guiana
SOUTH_AMERICA_COUNTRIES = ['BRA', 'COL', 'ARG', 'PER', 'VEN', 'CHL', 'ECU', 'BOL', 'PRY', 'URY', 'GUY', 'SUR']
TB_SA = WHO_TB[WHO_TB['iso3'].isin(SOUTH_AMERICA_COUNTRIES)]

In [29]:
# cropping data
TB_SA = TB_SA[[
  'country',
  'iso3',
  'year',
## population
  'e_pop_num',
## incidence
  'e_inc_num_lo',
  'e_inc_num',
  'e_inc_num_hi',
### hiv percentage in incident cases
#  'e_tbhiv_prct_lo',
#  'e_tbhiv_prct',
#  'e_tbhiv_prct_hi',
## incidence of hiv+tb
  'e_inc_tbhiv_num_lo',
  'e_inc_tbhiv_num', 
  'e_inc_tbhiv_num_hi',
## number of deaths
  'e_mort_num_lo',
  'e_mort_num', 
  'e_mort_num_hi',
## number of deaths, hiv only
  'e_mort_tbhiv_num_lo',
  'e_mort_tbhiv_num', 
  'e_mort_tbhiv_num_hi',
## new cases and reinfection by 100k pop
  'c_newinc_100k',
## historically "case detection rate", now denominated "treatment coverage"
  'c_cdr_lo',
  'c_cdr', 
  'c_cdr_hi',
]]

In [51]:
# reset cropped index
TB_SA.reset_index(drop=True, inplace=True)

In [54]:
# suriname 2007 data does not include new cases and treatment coverage
# so we propagate 2006 data as a simple estimate
TB_SA.fillna(method='ffill', inplace=True)

## Exportação

In [55]:
TB_SA.to_csv('data/tb_sa.csv', index=False)